In [12]:
from models import deep_strain_model
from datasets import base_dataset
from datasets.nifti_dataset import resample_nifti
from tensorflow.keras.optimizers import Adam
from scipy.ndimage import center_of_mass
from datasets.base_dataset import roll_and_pad_256x256_to_center_inv
from datasets.base_dataset import _roll2center_crop
from scipy.ndimage import gaussian_filter
import numpy as np
import SimpleITK as sitk
from utils.strain_from_motion import *
from utils.utils_aha import *
import nibabel as nib
from skimage.util import montage
import matplotlib.pyplot as plt
import os

########################## Mask and normalization ######################################


def normalize(x, axis=(0, 1, 2)):
    # normalize per volume (x,y,z) frame
    mu = x.mean(axis=axis, keepdims=True)
    sd = x.std(axis=axis, keepdims=True)
    return (x - mu) / (sd + 1e-8)


def get_mask(V, netS):
    nx, ny, nz, nt = V.shape

    M = np.zeros((nx, ny, nz, nt))
    v = V.transpose((2, 3, 0, 1)).reshape((-1, nx, ny))  # (nz*nt,nx,ny)
    v = normalize(v)
    for t in range(nt):
        for z in range(nz):
            m = netS(
                v[z * nt + t, nx // 2 - 64 : nx // 2 + 64, ny // 2 - 64 : ny // 2 + 64][
                    None, ..., None
                ]
            )
            M[nx // 2 - 64 : nx // 2 + 64, ny // 2 - 64 : ny // 2 + 64, z, t] += (
                np.argmax(m, -1).transpose((1, 2, 0)).reshape((128, 128))
            )
    return M

######################### Constants and arguments ######################################

DATASET_FOLDER = "ACDC"

class CarSON_options:
    def __init__(self):
        self.isTrain = False
        self.image_shape = (128, 128, 1)
        self.nlabels = 4
        self.pretrained_models_netS = "models/carson_Jan2021.h5"
        self.pretrained_models_netME = "models/carmen_Jan2021.h5"


class CarMEN_options:
    def __init__(self):
        self.isTrain = False
        self.volume_shape = (128, 128, 16, 1)
        self.pretrained_models_netS = "models/carson_Jan2021.h5"
        self.pretrained_models_netME = "models/carmen_Jan2021.h5"

In [15]:
opt = CarSON_options()
model = deep_strain_model.DeepStrain(Adam, opt=opt)
netS = model.get_netS()

out_folder = "data/training/"
patients = [f"patient{i:03d}" for i in range(1, 3)]

for patient in patients:
    print(f"Segmentation on patient {patient}")
    V_nifti = nib.load(os.path.join(DATASET_FOLDER, patient, f"{patient}_4d.nii.gz"))
    V_nifti_resampled = resample_nifti(
        V_nifti, order=1, in_plane_resolution_mm=1.25, number_of_slices=16
    )

    V = V_nifti_resampled.get_fdata()
    V = normalize(V, axis=(0, 1))
    M = get_mask(V, netS)

    center_resampled = center_of_mass(M[:, :, :, 0] == 2)
    V = base_dataset.roll_and_pad_256x256_to_center(x=V, center=center_resampled)
    M = base_dataset.roll_and_pad_256x256_to_center(x=M, center=center_resampled)
    center_resampled_256x256 = center_of_mass(M == 3)

    nifti_info = {
        "affine": V_nifti.affine,
        "affine_resampled": V_nifti_resampled.affine,
        "zooms": V_nifti.header.get_zooms(),
        "zooms_resampled": V_nifti_resampled.header.get_zooms(),
        "shape": V_nifti.shape,
        "shape_resampled": V_nifti_resampled.shape,
        "center_resampled": center_resampled,
        "center_resampled_256x256": center_resampled_256x256,
    }

    M = get_mask(V, netS)[128 - 64 : 128 + 64, 128 - 64 : 128 + 64]
    V = V[128 - 64 : 128 + 64, 128 - 64 : 128 + 64]

    M_nifti = base_dataset.convert_back_to_nifti(
        M, nifti_info, order=1, mode="nearest"
    )
    V_nifti = base_dataset.convert_back_to_nifti(
        V, nifti_info, order=1, mode="nearest"
    )

    M_nifti.to_filename(os.path.join(out_folder, f"{patient}_seg.nii.gz"))
    V_nifti.to_filename(os.path.join(out_folder, f"{patient}.nii.gz"))

Segmentation on patient patient001


FileNotFoundError: [Errno 2] No such file or directory: 'data/training/images/patient001_seg.nii.gz'

Data  that will be used for training is in ACDC folder

In [1]:
from models import deep_strain_model
from datasets import base_dataset
from datasets.nifti_dataset import resample_nifti
from tensorflow.keras.optimizers import Adam
from scipy.ndimage import center_of_mass
from datasets.base_dataset import roll_and_pad_256x256_to_center_inv
from datasets.base_dataset import _roll2center_crop
from scipy.ndimage import gaussian_filter
import numpy as np
import SimpleITK as sitk
from utils.strain_from_motion import *
from utils.utils_aha import *
import nibabel as nib
from skimage.util import montage
import matplotlib.pyplot as plt

##########################      Normalization     ######################################


def normalize(x, axis=(0, 1, 2)):
    # normalize per volume (x,y,z) frame
    mu = x.mean(axis=axis, keepdims=True)
    sd = x.std(axis=axis, keepdims=True)
    return (x - mu) / (sd + 1e-8)

######################### Constants and arguments ######################################

class CarMEN_options:
    def __init__(self):
        self.isTrain = True
        self.volume_shape = (128, 128, 16, 1)
        self.lambda_i = 0.1
        self.lambda_s = 0
        self.lambda_a = 0
